## Import der Bibliotheken, Definition der Messstellen und Download & Einlesen der Rohdaten

Die Messstelle des Pegelstandes, der vorhergesagt werden soll, und die Messstellen, die für die Vorhersage verwendet werden sollen, werden am Anfang des Codes definiert.
So kann das Skript leicht auch auf andere Ort angewendet werden. Die Klartextnamen der Messstellen stehen ebenfalls in den Download-Dateien.

Neben den im Seminar verwendeten Bibliotheken werden die Bibliotheken "urllib" für den Dowload von Dateien, "zipfile" für das Entpacken der Dowload-Dateien sowie "pyarrow" für ein effizienteres Einlesen von CSV-Dateien (https://pythonspeed.com/articles/pandas-read-csv-fast/) verwendet.

Autoren:
* Frederic Herrscher
* Julian Zilz (@julianzz98)

In [ ]:
# Import der benötigten Bibliotheken
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from datetime import datetime
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
from sklearn.preprocessing import StandardScaler
from tensorflow import keras
from keras.models import Sequential
from keras.layers import LSTM, Dense
from zipfile import ZipFile
import os
import urllib.request 
import pyarrow

# Pegelstand, der vorhergesagt werden soll
ms_prediction = '2747900000200'
# Messstellen, die für die Vorhersage verwendet werden sollen
ms_nieder = ['50040031','50040021','51050021','52080222'] # Niederschlagswerte
ms_temp = ['2741500000100','2747390000100','2747900000200'] # Temperaturswerte
ms_pegel = ['2741500000100','2743000000100','2747390000100','2747900000200','2741490000200','2746310000100','2746790000100','2744910000100','2742510000100','2742990000200','2741870000100'] # Pegelstände



In [ ]:
# Download-Ordner für die Dateien wird erstellt
dir = "./data/"
if not os.path.exists(dir):
    os.makedirs(dir)

# Die Dateien werden heruntergeladen und mithilfe der Bibliothek zipfile entpackt
# https://docs.python.org/3/library/urllib.request.html
# https://keras.io/examples/timeseries/timeseries_weather_forecasting/
uri_nieder = "https://www.opengeodata.nrw.de/produkte/umwelt_klima/wasser/oberflaechengewaesser/hygon/OpenHygon-Niederschlag-Bestand_CSV.zip"
urllib.request.urlretrieve(uri_nieder,dir + "Niederschlag.zip")
zip_file_nieder = ZipFile(dir + "Niederschlag.zip")
zip_file_nieder.extractall(dir)
txt_path_nieder = dir + "nieder_messwerte.txt"

uri_temp = "https://www.opengeodata.nrw.de/produkte/umwelt_klima/wasser/oberflaechengewaesser/hygon/OpenHygon-Wassertemperatur-Bestand_CSV.zip"
urllib.request.urlretrieve(uri_temp,dir + "Wassertemperatur.zip")
zip_file_temp = ZipFile(dir + "Wassertemperatur.zip")
zip_file_temp.extractall(dir)
txt_path_temp = dir + "temp_messwerte.txt"

uri_pegel = "https://www.opengeodata.nrw.de/produkte/umwelt_klima/wasser/oberflaechengewaesser/hygon/OpenHygon-Pegel-Bestand_CSV.zip"
urllib.request.urlretrieve(uri_pegel,dir + "Pegelstaende.zip")
zip_file_pegel = ZipFile(dir + "Pegelstaende.zip")
zip_file_pegel.extractall(dir)
txt_path_pegel = dir + "pegel_messwerte.txt"

In [ ]:
# DateParser, der nur gültige Datumsangaben importiert. So wird sichergestellt, dass der Type des Feldes immer 'datetime' ist
# https://pandas.pydata.org/docs/reference/api/pandas.to_datetime.html
def parse_date(x):
    try:
        return pd.to_datetime(x)
    except:
        return pd.NaT

# Import der benötigten CSV-Dateien
niederschlag_messwerte = pd.read_csv(txt_path_nieder, sep=';', parse_dates=['time'], date_parser=parse_date, header = 0, encoding= 'unicode_escape',engine="pyarrow")
#niederschlag_messwerte.info()
temperatur_messwerte = pd.read_csv(txt_path_temp, sep=';', parse_dates=['time'], date_parser=parse_date, header = 0, encoding= 'unicode_escape',engine="pyarrow")
#temperatur_messwerte.info()
pegelstand_messwerte = pd.read_csv(txt_path_pegel, sep=';', parse_dates=['time'], date_parser=parse_date, header = 0, encoding= 'unicode_escape',engine="pyarrow", index_col=['time'])
#pegelstand_messwerte.info()

## Data-Preprocessing

In [ ]:
# Die Umbenennung der Messwert-Felder hilft beim späteren Prozess
pegelstand_messwerte = pegelstand_messwerte.rename(columns={'value(cm)': 'Pegelstand'})
temperatur_messwerte = temperatur_messwerte.rename(columns={'value(cm)': 'Temperatur'})
niederschlag_messwerte = niederschlag_messwerte.rename(columns={'value(mm/h)': 'Niederschlag'})

In [ ]:
# Für den späteren Join ist es erforderlich, dass alle Felder den gleichen Typ haben
pegelstand_messwerte.station_no = pegelstand_messwerte.station_no.astype('string')
niederschlag_messwerte.station_no = niederschlag_messwerte.station_no.astype('string')
temperatur_messwerte.station_no = temperatur_messwerte.station_no.astype('string')
# pegelstand_messwerte.info()

In [ ]:
# Die Pegelstände werden von 15 Min. Daten auf 1h zusammengefasst (Resampling)
# https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.resample.html

pegelstand_messwerte_hour = pegelstand_messwerte.groupby('station_no').resample('1H').mean()
pegelstand_messwerte_hour = pegelstand_messwerte_hour.reset_index()
pegelstand_messwerte_hour.head()

In [ ]:
# Es wird eine leere Tabelle erstellt, die jedes Datum der Pegelstände nur einmal enthält. So können die anderen Daten nun darauf gejoined werden
# https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.drop_duplicates.html

wertetabelle = pegelstand_messwerte_hour
wertetabelle = wertetabelle['time'].drop_duplicates(keep='first')
wertetabelle = wertetabelle.to_frame() # wird als Series erstellt, muss daher umgewandelt werden

In [ ]:
# Nun werden die DataFrames auf die einzelnen Messstellen aus der Liste am Anfang gefiltert und mit dem neu erstellten DataFrame gejoined.
# https://www.youtube.com/watch?v=_gaAoJBMJ_Q&t=465s
# Da wir keine andere Möglichkeit gefunden haben, die Daten wie benötigt aufbereiten zu können, musste dieser Weg gegangen werden

for messstelle in ms_temp: # Es wird der eingangs definierte Array mit den Messstellen geloopt
    join_df = temperatur_messwerte.query('station_no == @messstelle') # Der Datensatz wird auf die Messstelle gefiltert
    wertetabelle = pd.merge(wertetabelle,join_df[['time','Temperatur']],on='time', how='left') # Nun werden die Daten gejoined
    wertetabelle = wertetabelle.rename(columns={'Temperatur': f'Temperatur_{messstelle}'}) # Die Umbenennung ist erforderlich, da sich sonst Spaltennamen wiederholen

for messstelle in ms_pegel:
    join_df = pegelstand_messwerte_hour.query('station_no == @messstelle')
    wertetabelle = pd.merge(wertetabelle,join_df[['time','Pegelstand']],on='time', how='left')
    wertetabelle = wertetabelle.rename(columns={'Pegelstand': f'Pegelstand_{messstelle}'})

for messstelle in ms_nieder:
    join_df = niederschlag_messwerte.query('station_no == @messstelle')
    wertetabelle = pd.merge(wertetabelle,join_df[['time','Niederschlag']],on='time', how='left')
    wertetabelle = wertetabelle.rename(columns={'Niederschlag': f'Niederschlag_{messstelle}'})

wertetabelle = wertetabelle.set_index('time')

# Der letzte Messzeitpunkt ist oft sehr unvollständig, daher löschen wir ihn (Die Daten werden von einigen Messstellen verspätet bereitgestellt)
wertetabelle = wertetabelle.drop(wertetabelle.index[-1])
wertetabelle.head()

In [ ]:
# Nun werden die NaN-Werte entfernt. Wir nutzen dafür eine Interpolation
# https://www.youtube.com/watch?v=EaGbS7eWSs0

wertetabelle = wertetabelle.replace('', np.nan).astype(float)
wertetabelle = wertetabelle.interpolate()

# Wenn der erste Wert der Tabelle NaN ist, wird er nicht durch die Interpolate-Methode ersetzt. Daher ersetzen wir ihn durch den nächsten Wert
wertetabelle = wertetabelle.bfill()
wertetabelle.isnull().sum()

In [ ]:
# Hier wird nun der Pegelstand, den wir vorhersagen, an die erste Stelle gepackt. Das benötigen wir später beim Split in X und Y Data
# https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.reindex.html

wertetabelle = wertetabelle.reindex(columns=[f'Pegelstand_{ms_prediction}'] + list(wertetabelle.columns.drop(f'Pegelstand_{ms_prediction}')))
# wertetabelle.to_csv('wertetabelle_after_prep.csv')
# wertetabelle.head()

## Scaling, Erstellung von Sequenzen und der Trainings- und Testdaten

In [ ]:
#Scaling der Daten mit StandardScaler (https://machinelearningmastery.com/standardscaler-and-minmaxscaler-transforms-in-python/)
scaler = StandardScaler()
scaler = scaler.fit(wertetabelle)
scaled_df = scaler.transform(wertetabelle)
# np.savetxt('scaled_data.csv', scaled_df, delimiter=',')
scaled_df.shape

In [ ]:
X = []
Y = []
predictX = []

n_future = 24 # Anzahl der Stunden, die wir in die Zukunft prognostizieren möchten
n_past = 96 # Anzahl der vergangenen Stunden, die wir verwenden möchten, um die Zukunft vorherzusagen

# Erstellung der Sequenzen von X und Y Daten
for i in range (n_past, len(scaled_df) - n_future + 1):
    X.append(scaled_df[i - n_past : i])
    Y.append(scaled_df[i : i+n_future, 0])

for i in range (n_past, len(scaled_df) + 1):
    predictX.append(scaled_df[i - n_past : i])

X, Y, predictX = np.array(X), np.array(Y), np.array(predictX)
print(X.shape)
print(Y.shape)
print(predictX.shape)
# np.savetxt("Y.csv", Y, delimiter=",")

# Erstellung der Trainings- und Testdaten
# https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html
trainX, testX, trainY, testY = train_test_split(X,Y,test_size=0.2, shuffle=True)

## Erstellen, Kompilieren und Trainieren des RNN

In [ ]:
# Erstellen des Modells mit LSTM und Dense-Layern
model = Sequential()
model.add(LSTM(128, input_shape=(trainX.shape[1], trainX.shape[2]), return_sequences=True))

model.add(LSTM(64, return_sequences=True, dropout=0.1))

model.add(LSTM(64, return_sequences=False, dropout=0.1))

model.add(Dense(trainY.shape[1]))

opt = keras.optimizers.Adam() # Festelegen des Optimizers
model.compile(optimizer = opt, loss="mse") # Kompilieren des Models
model.summary()

# Trainieren des Modells
history = model.fit(trainX, trainY, epochs=27, batch_size=8, validation_data=(testX,testY))

## Erstellen einer Vorhersage & Invertieren des Scalings

In [ ]:
# Erstellen einer Vorhersage mithilfe des trainierten Modells
forecast = model.predict(predictX)
forecast.shape

In [ ]:
# Invertierung des Skalierung. Da die Skalierung für jede Spalte der Inputdaten unterschiedlich ist, wird eine Hilfstabelle benötigt, um die korrekten Werte zu erhalten

invert_dataframe = pd.concat([wertetabelle.iloc[n_past-1:,0] for i in range(n_future)], axis=1)

invert_scaler = StandardScaler()
invert_scaler = invert_scaler.fit(invert_dataframe)
invert_scaled_dataframe = invert_scaler.transform(invert_dataframe)
#print(invert_scaled_dataframe)
print(invert_scaled_dataframe.shape)

y_pred = invert_scaler.inverse_transform(forecast)
y_pred


## Evaluation der vorhergesagten Daten

In [ ]:
# Für die Evaluation werden die vorhergesagten Daten mit den realen Daten verglichen
# Für den Vergleich wird jeweils der erste und letzte Vorhersagewert verwendet

real = wertetabelle.iloc[n_past:, 0].copy().reset_index()

pred_first = pd.DataFrame(y_pred[:,0], columns=["Vorhersage (first step)"]).reset_index(drop=True)

pred_last = pd.DataFrame(y_pred[:,n_future-1], columns=["Vorhersage (last step)"]).reset_index(drop=True)
pred_last.set_index(pd.Index(range(n_future-1,len(pred_first)+n_future-1)), inplace=True) # Index muss angepasst werden
pred_last

In [ ]:
# Es wird eine Ergebnistabelle erstellt
results = pd.merge(real, pred_first, how="outer", left_index=True, right_index=True)
results = pd.merge(results, pred_last, how="outer", left_index=True, right_index=True)

results = results.rename(columns={f'Pegelstand_{ms_prediction}': 'Realer Messwert'})

# Da keine Zeitwerte für die Zukunftswerte vorhanden sind, müssen diese mit der Pandas-Funtion TimeDelta hinzugefügt werden
# https://www.tutorialspoint.com/python_pandas/python_pandas_timedelta.htm
for i in range(0,n_future):
    results["time"] = results["time"].fillna(results["time"].shift() + pd.Timedelta(hours=1))
results = results.set_index("time")                      
# results.to_csv('results.csv')
results

In [ ]:
# Berechnen der Genauigkeit des ersten und letzten Zeitschrittes der Vorhersage mithilfe des Mean Squerad Errors und des Mean Absolute Percentage Errors

accuracy = results.iloc[n_future:-n_future].copy()

mape_first_step = mean_absolute_percentage_error(accuracy["Realer Messwert"], accuracy["Vorhersage (first step)"])
print(f"Der MAPE des first-step beträgt {mape_first_step}")

mape_last_step = mean_absolute_percentage_error(accuracy["Realer Messwert"], accuracy["Vorhersage (last step)"])
print(f"Der MAPE des last-step beträgt {mape_last_step}")

mse_first_step = mean_squared_error(accuracy["Realer Messwert"], accuracy["Vorhersage (first step)"])
print(f"Der MSE des first-step beträgt {mse_first_step}")

mse_last_step = mean_squared_error(accuracy["Realer Messwert"], accuracy["Vorhersage (last step)"])
print(f"Der MSE des last-step beträgt {mse_last_step}")

## Erstellung der Plots

In [ ]:
# Konfiguration des ersten Plots (Genauigekeit der Vorhersagen)
# https://www.activestate.com/resources/quick-reads/how-to-display-a-plot-in-python/

# Einstellen einer Farbpalette
# https://towardsdatascience.com/how-to-use-your-own-color-palettes-with-seaborn-a45bf5175146
colors = ["#001A61", "#E0572E", "#019417"]
sns.set_palette(sns.color_palette(colors))

fig, ax = plt.subplots(figsize=(22, 6))
ax.plot(results)
ax.legend(results.columns)

# Setze Achsenbeschriftungen und den Titel
ax.set_xlabel("Datum")
ax.set_ylabel("Pegelstand (cm)")
ax.set_title("Vergleich der realen Pegelstände mit den vorhergesagten Werten",fontsize=15, pad=25)
ax.tick_params(axis="both", direction="out", length=5, width=2, colors="black")
ax.legend(results.columns,loc="upper left", fontsize=10)

# Wochentage auf der X-Achse anzeigen
# https://matplotlib.org/stable/api/dates_api.html#matplotlib.dates.WeekdayLocator
ax.xaxis.set_major_formatter(mdates.DateFormatter('%d.%m.%Y'))
ax.xaxis.set_major_locator(mdates.WeekdayLocator(byweekday=1, interval=1))

ax.xaxis.set_tick_params(rotation=45, labelsize=10)
plt.xticks(rotation=0, size=8)
plt.grid(linestyle=':', linewidth='0.5')
plt.margins(x=0)

plt.savefig('result_plot.svg', format='svg')
plt.show()

In [ ]:
# Für den zweiten Plot (Vorhersage der nächsten n Zeitschritte) müssen die Tabellen zusammengefügt werden, um sie in einem Diagramm anzeigen zu können.
pred_future = y_pred[-1:]
pred_future = np.reshape(pred_future, (pred_future.shape[1], ))
pred_future = pd.DataFrame(pred_future, columns=["Vorhergesagter Messwert"])
pred_future.set_index(pd.Index(range(len(pred_first)-1,len(pred_first)+n_future-1)), inplace=True)
# print(pred_future)

results_future = pd.merge(real, pred_future, how="outer", left_index=True, right_index=True)
results_future = results_future.rename(columns={f'Pegelstand_{ms_prediction}': 'Realer Messwert'})

for i in range(0,n_future):
    results_future["time"] = results_future["time"].fillna(results_future["time"].shift() + pd.Timedelta(hours=1))
results_future = results_future.set_index("time")
results_future.loc[results_future.index[-n_future-1], 'Vorhergesagter Messwert'] = results_future.loc[results_future.index[-n_future-1], 'Realer Messwert']                  
results_future

In [ ]:
# Konfiguration des zweiten Plots

fig, ax = plt.subplots(figsize=(6, 6))
ax.plot(results_future.tail(60))
ax.legend(results_future.columns)

# Setze die Achsenbeschriftungen und den Titel
ax.set_xlabel("Datum")
ax.set_ylabel("Pegelstand (cm)")
ax.set_title("Vorhersage der kommenden 24 Stunden",fontsize=15, pad=25)
ax.tick_params(axis="both", direction="out", length=5, width=2, colors="black")
ax.legend(results_future.columns,loc="upper left", fontsize=10)

# https://matplotlib.org/stable/api/dates_api.html#matplotlib.dates.WeekdayLocator
ax.xaxis.set_major_formatter(mdates.DateFormatter('%d.%m.%Y'))
ax.xaxis.set_major_locator(mdates.DayLocator())
ax.xaxis.set_minor_locator(mdates.HourLocator())

ax.xaxis.set_tick_params(rotation=45, labelsize=10)
plt.xticks(rotation=0, size=8)
plt.grid(linestyle=':', linewidth='0.5')
plt.margins(x=0)

plt.savefig('result_plot_zoom.svg', format='svg')
plt.show()

## Optionaler Plot: Loss-Funktion verschiedener RNN-Konfigurationen

Um die Loss-Funktion verschiedener RNN-Konfigurationen zu vergleichen, müssen weitere Modelle kompiliert und trainiert werden. Wir haben den folgenden Code auskommentiert, um die Laufzeit des Skriptes zu verkürzen.

In [ ]:
# model = Sequential()
# model.add(LSTM(64, input_shape=(trainX.shape[1], trainX.shape[2]), return_sequences=True))

# model.add(LSTM(32, return_sequences=True, dropout=0.1))

# model.add(LSTM(32, return_sequences=False, dropout=0.1))

# model.add(Dense(trainY.shape[1]))

# opt = keras.optimizers.Adam()
# model.compile(optimizer = opt, loss="mse")
# model.summary()

# history2 = model.fit(trainX, trainY, epochs=27, batch_size=8, validation_data=(testX,testY))

# model = Sequential()
# model.add(LSTM(32, input_shape=(trainX.shape[1], trainX.shape[2]), return_sequences=True))

# model.add(LSTM(16, return_sequences=True, dropout=0.1))

# model.add(LSTM(8, return_sequences=False, dropout=0.1))

# model.add(Dense(trainY.shape[1]))

# opt = keras.optimizers.Adam()
# model.compile(optimizer = opt, loss="mse")
# model.summary()

# history3 = model.fit(trainX, trainY, epochs=27, batch_size=8, validation_data=(testX,testY))


In [ ]:
# # Erstellen des Plots der Loss-Funktionen 

# his1 = history.history
# his2 = history2.history
# his3 = history3.history

# train_loss1 = his1['loss']
# train_loss2 = his2['loss']
# train_loss3 = his3['loss']

# # X-Achse (Epochen)
# epochs = range(1, len(train_loss1) + 1)

# # Plotten des Trainings- und Validierungs-Losses beider Models
# plt.plot(epochs, train_loss1, label='Loss des Models mit 128/64/64 Neuronen')
# plt.plot(epochs, train_loss2, label='Loss des Models mit 64/32/32 Neuronen')
# plt.plot(epochs, train_loss3, label='Loss des Models mit 32/16/8 Neuronen')

# # Beschriftungen und Legende
# plt.title('Loss-Funktionen unterschiedlicher Modell-Konfigurationen', pad=15)
# plt.xlabel('Epochen')
# plt.ylabel('Loss')
# plt.legend()
# plt.grid(linestyle=':', linewidth='0.5')
# plt.margins(x=0)

# plt.savefig('loss_function_plot.svg', format='svg')
# plt.show()

